# "KPI Report"
# "KPI Report"

- toc:true
- branch: master
- badges: true
- comments: true
- author: Riki Hong
- categories: [Python, jupyter]

In [2]:
### 실적보고 RPA
import pandas as pd
import datetime

In [3]:
# 러닝시간 체크
stime = datetime.datetime.now()

d_today = datetime.date.today()
yr = d_today.strftime('%y')      # 현재 년도 가져오기
mn = d_today.month-1      # 실적월 가져오기
now_date = ("'" + yr + '. ' + str(mn+1) + '월')
past_date = ("'" + yr + '.' + str(mn) + '월')

In [ ]:

import docx
from openpyxl import load_workbook

#from docx.shared import Pt
#from docx.enum.text import WD_ALIGN_PARAGRAPH

data_df = pd.read_excel('D:\\2022년\\M-MAP TF\\KPI Report RPA\\법인별 KPI.xlsx', sheet_name = '지표별 Raw(월별실적)', usecols = "A:C, R:AE, AI:BG", header = 4)
data_df.rename(columns={'Unnamed: 0' : 'kpi','Unnamed: 1' : 'gbm', 'Unnamed: 2' : 'factory'}, inplace = True )

# 공백줄 제외
data_df = data_df[data_df['kpi'].isna() == False ]
# 띄어쓰기 제외
data_df['kpi'] = data_df['kpi'].str.replace(' ', '')

doc = docx.Document('D:\\2022년\\M-MAP TF\\KPI Report RPA\\★ DX부문 제조KPI.docx')


In [ ]:

# 법인값 삭제,Nan값 삭제
#gbm_df = data_df[data_df['factory'].isna() == True ]

gbm_list = ['DX', 'MX', 'VD', '생활가전', '네트워크' , '의료기기']
kpi_list = ['대당제조가공비', '인당생산대수', '자동화율', '공급능력지수', '운영Capa가동율', '총공정불량율', '제조인력', 'SOP실행율']
part_list = ['CNC', 'CNC', 'VD LCM', 'COMP', '-', '-']

#표넘버
t = 1
col = []
col_part = []
for g_lst in range(0, len(gbm_list)) :
    table = doc.tables[t]
    
    for row in table.rows : 
           
        if row.cells[1].paragraphs[0].text != '' :
            
            #표 KPI 넣기
            t_kpi = row.cells[1].paragraphs[0].text
            t_kpi = t_kpi.replace('(Set)', '')
            t_kpi = t_kpi.replace('(W/S)', '')
            
            # kpi와 사업부 조건에 맞는 것만 찾기
            df = data_df[(data_df['kpi'].str.contains(t_kpi)) & (data_df['gbm'].str.contains(gbm_list[g_lst]))]
            df.reset_index(drop = True, inplace = True)

            
            #부품 추가
            part_df = data_df[(data_df['kpi'].str.contains('대당제조가공비')) & (data_df['gbm'].str.contains(part_list[g_lst]))]
            part_df.reset_index(drop = True, inplace = True)
            
            if df.empty == False :
            
                col.append(df.iloc[0,3])
                col.append(df.loc[0,'누적목표'])
                col.append(df.loc[0,past_date + '누적'])
                col.append(df.iloc[0,1+mn])
                col.append(df.iloc[0,2+mn])
                col.append(df.iloc[0,3+mn])
                col.append(df.iloc[0,16])
        
                c = len(col)
                
                table.rows[1].cells[4].paragraphs[0].text = str(mn) +'월'
                table.rows[1].cells[c].paragraphs[0].text = str(mn) +'월'
                table.rows[1].cells[c-1].paragraphs[0].text = str(mn-1) +'월'
                table.rows[1].cells[c-2].paragraphs[0].text = str(mn-2) +'월'
                
 ####################################################################################################################################           
                if t_kpi == '대당제조가공비' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        
                        # 의료기기값의 경우는 소숫점 없이 표기
                        if col[i] > 10000 :
                            row.cells[i+2].paragraphs[0].text = '{:0,.0f}'.format(col[i])
                        else :
                            row.cells[i+2].paragraphs[0].text = '{:0,.2f}'.format(col[i])
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(df.loc[0,past_date + '누적'] / df.iloc[0,16] *100)
                    
                    # 전년비값
                    rate = (df.iloc[0,16] / df.iloc[0,3] -1) * 100                    
                    
                    if rate >= 0.5 :
                        rate = str('{:0,.1f}%'.format(rate)) + '↑'                        
                    elif rate <= -0.5 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + '↓'                                                
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate
                    row.cells[2].paragraphs[0].add_run('$')
                    
                    if part_df.empty == False :
                        
                        col_part.append(part_df.iloc[0,3])
                        col_part.append(part_df.loc[0,'누적목표'])
                        col_part.append(part_df.loc[0,past_date + '누적'])
                        col_part.append(part_df.iloc[0,1+mn])
                        col_part.append(part_df.iloc[0,2+mn])
                        col_part.append(part_df.iloc[0,3+mn])
                        col_part.append(part_df.iloc[0,16])

                        cp = len(col_part)

                        for ii in range(0, cp) :
                            # 실제 값 넣기
                            
                            row.cells[ii+2].paragraphs[1].text = '{:0,.2f}'.format(col_part[ii])                        
                        row.cells[cp+2].paragraphs[1].text = '{:0,.0f}%'.format(round(part_df.loc[0,past_date + '누적'],2) / round(part_df.iloc[0,16],2) *100)
                        
                        
                        # 달성율 값
                        rate = (round(part_df.iloc[0,16],2) / round(part_df.iloc[0,3],2) -1) * 100

                        if rate >= 0.5 :
                            rate = str('{:0,.1f}%'.format(rate)) + '↑'                        
                        elif rate <= -0.5 :
                            rate = str('{:0,.1f}%'.format(abs(rate))) + '↓'                                                
                        else :
                            rate = '-'         
                        

                        row.cells[cp+3].paragraphs[1].text = rate
                        row.cells[2].paragraphs[1].add_run('$')
                    
                    col_part = []
                    
#####################################################################################################################################

                elif t_kpi == '인당생산대수' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.1f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(round(df.iloc[0,16],1) / round(df.loc[0,past_date + '누적'],1) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],1) / round(df.iloc[0,3],1) -1) * 100
                    
                    if rate >= 0.5 :
                        rate = str('{:0,.1f}%'.format(rate)) + '↑'                        
                    elif rate <= -0.5 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + '↓'                                                
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('대')
                
                elif t_kpi == '자동화율' :
                    
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.1f}'.format(col[i])
                        
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(round(df.iloc[0,16],1) / round(df.loc[0,past_date + '누적'],1) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],1) - round(df.iloc[0,3],1))
                    
                    if rate >= 0.05 :
                        rate = str('{:0,.1f}%'.format(rate)) + 'p↑'                        
                    elif rate <= -0.05 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + 'p↓'                                                
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('%')                
                
                
                
                elif t_kpi == '공급능력지수' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.1f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.1f}%'.format(round(df.iloc[0,16],1) / round(df.loc[0,past_date + '누적'],1) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],1) - round(df.iloc[0,3],1))
                    
                    if rate >= 0.05 :
                        rate = str('{:0,.1f}%'.format(rate)) + 'p↑'                       
                    elif rate <= -0.05 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + 'p↓'                                               
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('%')
                    
                elif t_kpi == '운영Capa가동율' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.1f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(round(df.iloc[0,16],1) / round(df.loc[0,past_date + '누적'],1) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],1) - round(df.iloc[0,3],1))
                    
                    if rate >= 0.05 :
                        rate = str('{:0,.1f}%'.format(rate)) + 'p↑'
                    elif rate <= -0.05 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + 'p↓'                        
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('%')
                                
                elif t_kpi == '총공정불량율' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.0f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(round(df.loc[0,past_date + '누적'],0) / round(df.iloc[0,16],0) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],0) / round(df.iloc[0,3],0) -1) * 100
                    
                    if rate >= 0.5 :
                        rate = str('{:0,.1f}%'.format(rate)) + '↑'            
                    elif rate <= -0.5 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + '↓'                                                
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('ppm')      
                    
                elif t_kpi == '제조인력' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.0f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.0f}%'.format(round(df.loc[0,past_date + '누적'],0) / round(df.iloc[0,16],0) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],0) / round(df.iloc[0,3],0) -1) * 100
                    
                    if rate >= 0.5 :
                        rate = str('{:0,.1f}%'.format(rate)) + '↑'                        
                    elif rate <= -0.5 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + '↓'                        
                    else :
                        rate = '-'
                        
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('명')      
                    
                    
                elif t_kpi == 'SOP실행율' :
                    for i in range(0, c) :
                        # 실제 값 넣기
                        row.cells[i+2].paragraphs[0].text = '{:0,.1f}'.format(col[i])
                    
                    # 달성율 값
                    row.cells[c+2].paragraphs[0].text = '{:0,.1f}%'.format(round(df.iloc[0,16],1) / round(df.loc[0,past_date + '누적'],1) *100)
                    
                    # 전년비
                    rate = (round(df.iloc[0,16],1) - round(df.iloc[0,3],1))
                    
                    if rate >= 0.05 :
                        rate = str('{:0,.1f}%'.format(rate)) + 'p↑'                       
                    elif rate <= -0.05 :
                        rate = str('{:0,.1f}%'.format(abs(rate))) + 'p↓'                                               
                    else :
                        rate = '-'
                    
                    row.cells[c+3].paragraphs[0].text = rate                    
                    row.cells[2].paragraphs[0].add_run('%')
                    
                col = []
                

    t = t+1
            
#             row.cells[2].paragraphs[0].text = '{:0,.2f}'.format(df.iloc[0,3])
#             row.cells[3].paragraphs[0].text = df.iloc[0,'누적목표']


# 타이틀 수정
doc.tables[0].rows[0].cells[0].paragraphs[1].text = past_date + ' 제조 KPI 실적'

# 표 컬럼 수정(3개월로 변경되면 10으로 바꿔야함!)################################
tc = 10

# 본문수정
c = 0

for p in doc.paragraphs: 
    
    if c == 17 :
        p.add_run(now_date)
        
    # DX 대당제조가공비
    elif c == 21 :
        acc_target = doc.tables[1].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[1].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[1].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 22 :
        last_year = doc.tables[1].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[1].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )

    # DX 부품          
    elif c == 23 :
        acc_target = doc.tables[1].rows[3].cells[4].paragraphs[1].text
        acc_result = doc.tables[1].rows[3].cells[tc-2].paragraphs[1].text
        pct = doc.tables[1].rows[3].cells[tc-1].paragraphs[1].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ', 실적 ' + acc_result + '(달성율 '+ pct + ')')
    
    # DX 인당생산대수
    elif c == 24 :
        acc_target = doc.tables[1].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[1].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[1].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '대 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 25 :
        last_year = doc.tables[1].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[1].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # DX 자동화율
    elif c == 26 :
        acc_target = doc.tables[1].rows[5].cells[4].paragraphs[0].text
        acc_result = doc.tables[1].rows[5].cells[tc-2].paragraphs[0].text
        pct = doc.tables[1].rows[5].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '% 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 27 :
        last_year = doc.tables[1].rows[5].cells[2].paragraphs[0].text
        ypct = doc.tables[1].rows[5].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )        
    
    ############################################################################################
    
    # MX 대당제조가공비
    elif c == 31 :
        acc_target = doc.tables[2].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[2].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[2].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 32 :
        last_year = doc.tables[2].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[2].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )

    # MX 부품          
    elif c == 33 :
        acc_target = doc.tables[2].rows[3].cells[4].paragraphs[1].text
        acc_result = doc.tables[2].rows[3].cells[tc-2].paragraphs[1].text
        pct = doc.tables[2].rows[3].cells[tc-1].paragraphs[1].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ', 실적 ' + acc_result + '(달성율 '+ pct + ')')
    
    # MX 인당생산대수
    elif c == 34 :
        acc_target = doc.tables[2].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[2].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[2].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '대 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 35 :
        last_year = doc.tables[2].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[2].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # MX 자동화율
    elif c == 36 :
        acc_target = doc.tables[2].rows[5].cells[4].paragraphs[0].text
        acc_result = doc.tables[2].rows[5].cells[tc-2].paragraphs[0].text
        pct = doc.tables[2].rows[5].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '% 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 37 :
        last_year = doc.tables[2].rows[5].cells[2].paragraphs[0].text
        ypct = doc.tables[2].rows[5].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )    
        

    ############################################################################################
    
    # vd 대당제조가공비
    elif c == 40 :
        acc_target = doc.tables[3].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[3].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[3].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 41 :
        last_year = doc.tables[3].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[3].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )

    # vd 부품          
    elif c == 42 :
        acc_target = doc.tables[3].rows[3].cells[4].paragraphs[1].text
        acc_result = doc.tables[3].rows[3].cells[tc-2].paragraphs[1].text
        pct = doc.tables[3].rows[3].cells[tc-1].paragraphs[1].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ', 실적 ' + acc_result + '(달성율 '+ pct + ')')
    
    # vd 인당생산대수
    elif c == 43 :
        acc_target = doc.tables[3].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[3].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[3].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '대 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 44 :
        last_year = doc.tables[3].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[3].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # vd 자동화율
    elif c == 45 :
        acc_target = doc.tables[3].rows[5].cells[4].paragraphs[0].text
        acc_result = doc.tables[3].rows[5].cells[tc-2].paragraphs[0].text
        pct = doc.tables[3].rows[5].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '% 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 46 :
        last_year = doc.tables[3].rows[5].cells[2].paragraphs[0].text
        ypct = doc.tables[3].rows[5].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )    
        

        
    ############################################################################################
    
    # 가전 대당제조가공비
    elif c == 49 :
        acc_target = doc.tables[4].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[4].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[4].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 50:
        last_year = doc.tables[4].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[4].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )

    # 가전 부품          
    elif c == 51 :
        acc_target = doc.tables[4].rows[3].cells[4].paragraphs[1].text
        acc_result = doc.tables[4].rows[3].cells[tc-2].paragraphs[1].text
        pct = doc.tables[4].rows[3].cells[tc-1].paragraphs[1].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ', 실적 ' + acc_result + '(달성율 '+ pct + ')')
    
    # 가전 인당생산대수
    elif c == 52 :
        acc_target = doc.tables[4].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[4].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[4].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '대 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 53 :
        last_year = doc.tables[4].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[4].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # 가전 자동화율
    elif c == 54:
        acc_target = doc.tables[4].rows[5].cells[4].paragraphs[0].text
        acc_result = doc.tables[4].rows[5].cells[tc-2].paragraphs[0].text
        pct = doc.tables[4].rows[5].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '% 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 55 :
        last_year = doc.tables[4].rows[5].cells[2].paragraphs[0].text
        ypct = doc.tables[4].rows[5].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )  
        
        
    ############################################################################################
    
    # N/W 대당제조가공비
    elif c == 58 :
        acc_target = doc.tables[5].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[5].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[5].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 59:
        last_year = doc.tables[5].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[5].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # N/W 인당생산대수
    elif c == 60 :
        acc_target = doc.tables[5].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[5].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[5].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '대 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 61 :
        last_year = doc.tables[5].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[5].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # N/W 자동화율
    elif c == 62 :
        acc_target = doc.tables[5].rows[5].cells[4].paragraphs[0].text
        acc_result = doc.tables[5].rows[5].cells[tc-2].paragraphs[0].text
        pct = doc.tables[5].rows[5].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '% 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 63 :
        last_year = doc.tables[5].rows[5].cells[2].paragraphs[0].text
        ypct = doc.tables[5].rows[5].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )   
        
    ############################################################################################
    
    # 의료기기 대당제조가공비
    elif c == 67:
        acc_target = doc.tables[6].rows[3].cells[4].paragraphs[0].text
        acc_result = doc.tables[6].rows[3].cells[tc-2].paragraphs[0].text
        pct = doc.tables[6].rows[3].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 $' + acc_target + ' 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 68 :
        last_year = doc.tables[6].rows[3].cells[2].paragraphs[0].text
        ypct = doc.tables[6].rows[3].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    
    # 의료기기 제조인력
    elif c == 69 :
        acc_target = doc.tables[6].rows[4].cells[4].paragraphs[0].text
        acc_result = doc.tables[6].rows[4].cells[tc-2].paragraphs[0].text
        pct = doc.tables[6].rows[4].cells[tc-1].paragraphs[0].text
        p.add_run(past_date + ' 누적목표 ' + acc_target + '명 比 실적 ' + acc_result + '(달성율 '+ pct + ')')
    elif c == 70 :
        last_year = doc.tables[6].rows[4].cells[2].paragraphs[0].text
        ypct = doc.tables[6].rows[4].cells[tc].paragraphs[0].text
        p.add_run(str(int(yr)-1) + '년실적 ' + last_year + ' 比 ' + ypct )
    

        
    c = c+1
    
    
doc.save('D:\\2022년\\M-MAP TF\\KPI Report RPA\\★ ' + past_date + ' DX부문 제조KPI_F.docx')

etime = datetime.datetime.now()

print(etime - stime)


for x, paragraph in enumerate(doc.paragraphs) :
    if paragraph.text != '' :
        print(str(x) + ' : ' + paragraph.text)
